<a href="https://colab.research.google.com/github/dzakyfaishalariq/Catatan-Ai-Developer/blob/main/Learning_LLM_and_Generatif_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Learning LLM Generatif AI

## 1. The Basics LangChain ⛓🐦

**Explanation of langchain**

langchain is a framework for building Artificial intelligence from the foundation of large language model (LLM). There are several tool that will be used when using and runing this (LLM).

The tools :
- LangChain
- groq
- python-dotenv
- getpass

### Installation and Setup

In [ ]:
! pip install langchain groq langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.5/127.5 kB 4.1 MB/s eta 0:00:00


### Connection to the GROQ model

#### Import library

In [ ]:
from langchain.chat_models import init_chat_model
import getpass
import os
import re

#### Inisialisasi Models

In [ ]:
llm = init_chat_model(
    "deepseek-r1-distill-llama-70b",
    model_provider="groq",
    temperature=0.7,
    groq_api_key=getpass.getpass("Enter your GROQ API key: ")
)

Enter your GROQ API key: ··········


#### Create promp models

In [ ]:
respons = llm.invoke("""
  Here are specific instructions for this model:
  1. Do not show internal thinking processes in the <think> format
  2. just give the final answer directly
  3. the output format must be clean without markup

  questionL: What is langchain?
""")

#### Show to response

In [ ]:
print(respons.content)

<think>
Okay, I need to figure out what LangChain is. I've heard the term before, but I'm not exactly sure. Let me start by breaking down the word: "Lang" probably refers to language, and "Chain" might imply a sequence or series. So, maybe it's something related to processing language in a sequence.

I remember hearing about AI and machine learning tools. Perhaps LangChain is a tool or framework in that space. I think it might be related to large language models, like GPT. Maybe it's a way to chain together different AI models or processes.

Wait, I think I've heard it mentioned in the context of AI applications. Maybe it's a framework that allows developers to build applications using multiple language models or combine different AI services. That makes sense because "chain" could mean linking several components together.

I should check if it's open-source. Many AI tools are open-source these days. If LangChain is open-source, that would mean it's accessible for developers to use and

#### Delete Unnecessary Sentences

In [ ]:
def remove_think_tags(response):
  return re.sub(r"<think>.*?</think>", "", response, flags=re.DOTALL).strip()

In [ ]:
clean_response = remove_think_tags(respons.content)

In [ ]:
print(clean_response)

LangChain is an open-source framework designed to help developers build applications by chaining together multiple language models and AI services, enabling more complex and powerful natural language processing tasks.


### Simpel Chains

**Explanation** : At this stage, the use of prompt template is needed as a framwork for preparing user questions to make them clearer.  

In [ ]:
from langchain.prompts import ChatPromptTemplate

In [ ]:
# promting template
prompt = ChatPromptTemplate.from_template(
    "Jelaskan konsep {concept} dalam bidang {field} dengan bahasa yang mudah dipahami dan menggunakan bahasa indonesia"
)

# create simpel chain
chain = prompt | llm

In [ ]:
concept = input("Input for concept: ")
field = input("Input for field: ")

response = chain.invoke({"concept": concept, "field": field})
print(remove_think_tags(response.content))

Input for concept: Deep Learning
Input for field: Biologi
**Pengantar Deep Learning dalam Biologi**

Deep Learning adalah bagian dari Kecerdasan Buatan (AI) yang menggunakan jaringan saraf tiruan, mirip dengan lapisan sel saraf otak manusia, untuk mengenali pola dalam data. Seperti otak kita yang belajar dari pengalaman, komputer dengan Deep Learning dapat memproses informasi kompleks dan meningkatkan kemampuannya seiring waktu.

**Aplikasi Deep Learning dalam Biologi**

1. **Genomik**: Deep Learning membantu menganalisis data genetik untuk mengidentifikasi gen yang terkait penyakit. Contohnya, memprediksi ekspresi gen berdasarkan data DNA.

2. **Protein dan Struktur Biomolekul**: Teknologi seperti AlphaFold menggunakan Deep Learning untuk memprediksi bagaimana protein akan terlipat, informasi penting untuk memahami bagaimana mereka bekerja dalam tubuh.

3. **Kedokteran dan Imaging**: Dalam bidang medis, Deep Learning diterapkan untuk menganalisis gambar MRI atau X-ray untuk mendeteksi

# Intermediet Leavel: RAG(Retrieval-Augmented-Generation)

**Eksplanation** : RAG is a function to create a model that knows the document file that we provide so that it can be read and find out the information in it.

In [ ]:
!pip install --upgrade pip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 40.7 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [ ]:
!pip install langchain-community faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.3 MB/s eta 0:00:00


## Load document

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

In [ ]:
# load documen base website
loader = WebBaseLoader("https://aws.amazon.com/id/what-is/artificial-intelligence/")
docs = loader.load()

In [ ]:
# split documen
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(docs)

In [ ]:
# create vector store
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
vectorstore = FAISS.from_documents(texts, embeddings)

In [ ]:
vectorstore.save_local("ai_faiss_index")

In [ ]:
# Implementation RAG
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain.prompts import ChatPromptTemplate

In [ ]:
# load vectors store
vectorstore = FAISS.load_local("/content/ai_faiss_index", embeddings, allow_dangerous_deserialization=True)

In [ ]:
retriever = vectorstore.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7ab5c12a6b50>, search_kwargs={})

In [ ]:
# promting
prompt = ChatPromptTemplate.from_template(
    """
    Jawablah pertanyaan berikut berdasarkan hanya pada konteks yang diberikan:
    <konteks>
    {context}
    </konteks>

    Pertanyaan: {input}
    """
)

# document chain
document_chain = create_stuff_documents_chain(llm, prompt)


In [ ]:
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, template='\n    Jawablah pertanyaan berikut berdasarkan hanya pada konteks yang diberikan:\n    <konteks>\n    {context}\n    </konteks>\n\n    Pertanyaan: {input}\n    '), additional_kwargs={})])
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7ab728b6c310>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7ab728ba2310>, model_name='deepseek-r1-distill-llama-70b', model_kwargs={}, groq_api_key=SecretStr('**********'))
| StrOutputParser(), kwargs={}, config={'run_name': 'stuff_documents_chain'}, config_factories=[])

In [ ]:
# create retrival chain
retrival_chain = create_retrieval_chain(retriever, document_chain)

In [ ]:
retrival_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7ab5c12a6b50>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, template='\n    Jawablah pertanyaan berikut berdasarkan hanya pada konteks yang diberikan:\n    <konteks>\n    {context}\n    </konteks>\n\n

In [ ]:
# run rag
response = retrival_chain.invoke(
    {
        "input":"Platform apa yang dijelaskan pada artikel ini untuk mengelolah kecerdasan buatan?"
    }
)
print(remove_think_tags(response['answer']))

Platform yang dijelaskan dalam artikel untuk mengelola kecerdasan buatan adalah **AWS (Amazon Web Services)**.


## Load multipel documents

In [ ]:
from tqdm import tqdm
import time

In [ ]:
# list Url for load documents
urls = [
    "https://www.detik.com/hikmah/doa-dan-hadits/d-6791270/30-hadits-pendek-dan-artinya-mengandung-makna-baik-dalam-kehidupan",
    "https://id.wikipedia.org/wiki/Sunah",
    "https://fai.umsu.ac.id/islami/5-rukun-islam-pengertian-dan-maknanya/"
]

In [ ]:
# Load documents form urls with tracking progress
print("start loading documents")
docs = []
for url in tqdm(urls, desc="loading websites"):
  try:
    loader = WebBaseLoader(url)
    docs.extend(loader.load())
    time.sleep(1)
  except Exception as e:
    print(f"Error loading website {url}: {e}")

print(f"total documents: {len(docs)}")


start loading documents


loading websites: 100%|██████████| 3/3 [00:10<00:00,  3.59s/it]

total documents: 3


In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False
)
print("Processing splitting documents")
texts = text_splitter.split_documents(docs)
print(f"total chunks producks: {len(texts)}")

Processing splitting documents
total chunks producks: 45


In [ ]:
#inisialisasi embedings with custum configurasi
embedings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
    model_kwargs={"device": "cpu"},
    encode_kwargs={"normalize_embeddings": True}
)

<ipython-input-18-a6f024771f3b>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedings = HuggingFaceEmbeddings(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.89k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# create vector store with optimisasi
print("create vector store...")
vectorstore_multi_docs = FAISS.from_documents(
    texts,
    embedings,
    normalize_L2=True,
)

create vector store...


In [ ]:
vectorstore_multi_docs

In [ ]:
import datetime
version = datetime.datetime.now().strftime("%Y%m%d")
save_path = f"ai_faiss_index_{version}"
print(f"save vector store to {save_path}")
vectorstore_multi_docs.save_local(save_path)

save vector store to ai_faiss_index_20250506


In [ ]:
# verifikasi result
print("\nVerifikasi")
print(f"- Total original documents : {len(docs)}")
print(f"- Total chunks producks : {len(texts)}")
print(f"- Dimensi embedings : {embedings.client.get_sentence_embedding_dimension()}")


Verifikasi
- Total original documents : 3
- Total chunks producks : 45
- Dimensi embedings : 384


In [ ]:
# load vector store
retriever = vectorstore_multi_docs.as_retriever()

In [ ]:
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7f3b99d58d90>, search_kwargs={})

In [ ]:
create_template_promp = ChatPromptTemplate.from_template(
    """
    Jawablah pertanyaan berikut berdasarkan hanya pada konteks yang diberikan jika tidak ada di konteks maka katakan 'saya tidak mengerti maksut pertanyaan anda':
    <konteks>
    {context}
    </konteks>

    Pertanyaan: {input}
    """
)

In [ ]:
document_chain = create_stuff_documents_chain(llm, create_template_promp)

In [ ]:
retrival_chain = create_retrieval_chain(retriever, document_chain)

In [ ]:
# run rag
Respons = retrival_chain.invoke({
    "input":"Coba ringkas ketiga sumber document yang sudah saya masukan!"
})
repons = remove_think_tags(Respons['answer'])
print(repons)

Tiga sumber dokumen yang disampaikan dalam konteks tersebut adalah:

1. **Daftar Isi**: Merupakan bagian yang mencakup etimologi, dalil, referensi, dan pranala luar, yang mungkin berasal dari suatu dokumen atau bab dalam buku yang membahas topik tersebut.

2. **Sunah dalam 81 Bahasa**: Sumber ini membahas tentang sunah dalam Islam dengan tersedia dalam 81 bahasa, menunjukkan sumber daya yang luas dan multibahasa.

3. **Hadits Pendek tentang Kehidupan Sehari-hari**: Sumber ini mengutip buku "Hadits-Hadits Pendek Mudah Dihafal" karya Hj. Siti Alfiah, yang mengumpulkan hadits-hadits pendek yang mudah dihafal dan relevan untuk kehidupan sehari-hari.
